# Final model
Here the best combination of predictor and dataset features will be determined and validated.

## Model selection with feature combinations
We evaluate multiple feature sets (baseline, holidays, Fourier, trend, and their combination) together with different predictors. Each predictor is fine-tuned via a small hyperparameter grid, and the best-performing combination on the validation split is returned.

In [15]:
from __future__ import annotations
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS, SeasonalNaive
from statsforecast.models import AutoARIMA, AutoETS, SeasonalNaive
import inspect

DATA_DIR = Path("..") / "data" / "processed_data"
FEATURE_KEYS = ["none", "holidays", "fourier", "trend", "fourier+trend+holidays"]
SPLITS = ["train", "val"]
FREQ = "D"

def load_feature_sets():
    features = {k: {} for k in FEATURE_KEYS}
    future = {k: {} for k in FEATURE_KEYS}
    for key in FEATURE_KEYS:
        for split in SPLITS:
            path = DATA_DIR / f"{key}_{split}.parquet"
            features[key][split] = pl.read_parquet(path) if path.exists() else None
            future_path = DATA_DIR / f"{key}_{split}_future.parquet"
            future[key][split] = pl.read_parquet(future_path) if future_path.exists() else None
    return features, future

def prepare_target(df: pl.DataFrame) -> pd.DataFrame:
    return df.select(["unique_id", "ds", "y"]).to_pandas()

features, features_future = load_feature_sets()
features

{'none': {'train': shape: (19_180, 3)
  ┌────────────┬──────────────┬────────┐
  │ ds         ┆ unique_id    ┆ y      │
  │ ---        ┆ ---          ┆ ---    │
  │ date       ┆ str          ┆ f64    │
  ╞════════════╪══════════════╪════════╡
  │ 2020-07-01 ┆ "Bubble tea" ┆ 2012.0 │
  │ 2020-07-02 ┆ "Bubble tea" ┆ 2085.0 │
  │ 2020-07-03 ┆ "Bubble tea" ┆ 2204.0 │
  │ 2020-07-04 ┆ "Bubble tea" ┆ 2119.0 │
  │ 2020-07-05 ┆ "Bubble tea" ┆ 2176.0 │
  │ …          ┆ …            ┆ …      │
  │ 2025-09-26 ┆ "Tea"        ┆ 1206.0 │
  │ 2025-09-27 ┆ "Tea"        ┆ 1074.0 │
  │ 2025-09-28 ┆ "Tea"        ┆ 1222.0 │
  │ 2025-09-29 ┆ "Tea"        ┆ 1211.0 │
  │ 2025-09-30 ┆ "Tea"        ┆ 1220.0 │
  └────────────┴──────────────┴────────┘,
  'val': shape: (310, 3)
  ┌────────────┬──────────────┬────────┐
  │ ds         ┆ unique_id    ┆ y      │
  │ ---        ┆ ---          ┆ ---    │
  │ date       ┆ str          ┆ f64    │
  ╞════════════╪══════════════╪════════╡
  │ 2025-10-01 ┆ "Bubble tea" ┆ 11

In [16]:
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [19]:
def get_horizon(df_future: pl.DataFrame | None, val_df: pl.DataFrame) -> int:
    if df_future is not None:
        first_id = df_future["unique_id"][0]
        return df_future.filter(pl.col("unique_id") == first_id).height
    first_id = val_df["unique_id"][0]
    return val_df.filter(pl.col("unique_id") == first_id).height

MODEL_GRID = {
    "AutoARIMA": {
        "constructor": AutoARIMA,
        "params": {
            "season_length": [7],
            "stepwise": [True],
            "approximation": [True, False],
        },
    },
    "AutoETS": {
        "constructor": AutoETS,
        "params": {
            "season_length": [7],
            "model": ["ZZZ"],
        },
    },
    "SeasonalNaive": {
        "constructor": SeasonalNaive,
        "params": {
            "season_length": [7, 14],
        },
    },
    # RandomForestRegressor handled separately (not a StatsForecast model)
    "RandomForestRegressor": {
        "constructor": RandomForestRegressor,
        "params": {
            "n_estimators": [200, 400, 600],
            "max_depth": [None, 10, 20],
            "random_state": [42],
            "n_jobs": [-1],
        },
    },
}


def _fit_with_exog(sf: StatsForecast, df: pd.DataFrame):
    return sf.fit(df=df)

def _predict_with_exog(fitted: StatsForecast, h: int):
    return fitted.predict(h=h)

def evaluate_feature_model(feature_key: str):
    train = features[feature_key].get("train")
    val = features[feature_key].get("val")
    val_future = features_future[feature_key].get("val")
    if train is None or val is None:
        return []
    y_train = prepare_target(train)
    y_val = prepare_target(val)
    h = get_horizon(val_future, val)

    results = []
    for model_name, cfg in MODEL_GRID.items():
        for params in ParameterGrid(cfg["params"]):
            # RandomForest branch uses all non-(unique_id, ds, y) columns already in the dataset
            if model_name == "RandomForestRegressor":
                train_df = train.to_pandas()
                val_df = val.to_pandas()
                feature_cols = [c for c in train_df.columns if c not in ["unique_id", "ds", "y"]]
                if not feature_cols:
                    print(f"Skipping RandomForest for {feature_key}: no feature columns available")
                    continue

                train_df = train_df[["unique_id", "ds", "y", *feature_cols]].fillna("")
                val_df = val_df[["unique_id", "ds", "y", *feature_cols]].fillna("")

                cat_cols = [c for c in feature_cols if train_df[c].dtype == "object"]
                train_X = pd.get_dummies(train_df[feature_cols], columns=cat_cols, drop_first=False)
                val_X = pd.get_dummies(val_df[feature_cols], columns=cat_cols, drop_first=False)

                cols = sorted(set(train_X.columns) | set(val_X.columns))
                train_X = train_X.reindex(columns=cols, fill_value=0)
                val_X = val_X.reindex(columns=cols, fill_value=0)

                rf = cfg["constructor"](**params)
                rf.fit(train_X, train_df["y"])
                preds = rf.predict(val_X)
                merged = val_df.copy()
                merged["yhat"] = preds
                mae = mean_absolute_error(merged["y"], merged["yhat"])
                rmse = root_mean_squared_error(merged["y"], merged["yhat"])
                smape = np.mean(200 * np.abs(merged["yhat"] - merged["y"]) / (np.abs(merged["y"]) + np.abs(merged["yhat"]) + 1e-8))
                r2 = r2_score(merged["y"], merged["yhat"])
                results.append({
                    "feature_set": feature_key,
                    "model": model_name,
                    "params": params,
                    "used_exog": True,
                    "mae": mae,
                    "rmse": rmse,
                    "smape": smape,
                    "r2": r2,
                })
                continue

            model = cfg["constructor"](**params)
            sf = StatsForecast(models=[model], freq=FREQ, n_jobs=-1)
            fitted = _fit_with_exog(sf, y_train)
            fcst = _predict_with_exog(fitted, h)
            yhat_col = [c for c in fcst.columns if c not in ("unique_id", "ds")][0]
            preds = fcst.rename(columns={yhat_col: "yhat"})
            merged = y_val.merge(preds, on=["unique_id", "ds"], how="inner")
            mae = mean_absolute_error(merged["y"], merged["yhat"])
            rmse = root_mean_squared_error(merged["y"], merged["yhat"])
            smape = np.mean(200 * np.abs(merged["yhat"] - merged["y"]) / (np.abs(merged["y"]) + np.abs(merged["yhat"]) + 1e-8))
            r2 = r2_score(merged["y"], merged["yhat"])
            results.append({
                "feature_set": feature_key,
                "model": model_name,
                "params": params,
                "used_exog": False,
                "mae": mae,
                "rmse": rmse,
                "smape": smape,
                "r2": r2,
            })
    return results

In [20]:
all_results: list[dict] = []
for feature_key in FEATURE_KEYS:
    all_results.extend(evaluate_feature_model(feature_key))

if not all_results:
    raise RuntimeError("No feature/model results were produced. Ensure processed parquet files exist.")

results_df = pd.DataFrame(all_results).sort_values(["rmse", "mae"]).reset_index(drop=True)
display(results_df.head(10))

best_combo = results_df.iloc[0]
best_combo

Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available
Skipping RandomForest for none: no feature columns available


,feature_set,model,params,used_exog,mae,rmse,smape,r2
0,none,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246,0.980389
1,holidays,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246,0.980389
2,fourier,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246,0.980389
3,trend,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246,0.980389
4,fourier+trend+holidays,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246,0.980389
5,none,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107,0.979113
6,holidays,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107,0.979113
7,fourier,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107,0.979113
8,trend,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107,0.979113
9,fourier+trend+holidays,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107,0.979113


feature_set                                    none
model                                       AutoETS
params         {'model': 'ZZZ', 'season_length': 7}
used_exog                                     False
mae                                       78.507702
rmse                                     169.319382
smape                                     19.019246
r2                                         0.980389
Name: 0, dtype: object

## Test set evaluation
Load test split, retrain best model on train+val, and generate predictions against baseline.

In [21]:
# Load test split
SPLITS_TEST = ["train", "val", "test"]

def load_all_splits(feature_key: str):
    splits_data = {}
    futures_data = {}
    for split in SPLITS_TEST:
        path = DATA_DIR / f"{feature_key}_{split}.parquet"
        splits_data[split] = pl.read_parquet(path) if path.exists() else None
        future_path = DATA_DIR / f"{feature_key}_{split}_future.parquet"
        futures_data[split] = pl.read_parquet(future_path) if future_path.exists() else None
    return splits_data, futures_data

best_feature = best_combo["feature_set"]
best_model_name = best_combo["model"]
best_params = best_combo["params"]
used_exog = best_combo["used_exog"]

print(f"Best: {best_model_name} with {best_feature} features")
print(f"Params: {best_params}")
print(f"Used exog: {used_exog}")

splits, futures = load_all_splits(best_feature)
splits.keys()

Best: AutoETS with none features
Params: {'model': 'ZZZ', 'season_length': 7}
Used exog: False


dict_keys(['train', 'val', 'test'])

In [22]:
# Combine train+val for final training
y_train_val = pd.concat([
    prepare_target(splits["train"]),
    prepare_target(splits["val"])
], ignore_index=True).sort_values(["unique_id", "ds"]).reset_index(drop=True)

y_test = prepare_target(splits["test"])

# Feature columns already present in dataset (non-unique_id/ds/y)
train_full = splits["train"].to_pandas()
val_full = splits["val"].to_pandas()
test_full = splits["test"].to_pandas()
feature_cols = [c for c in train_full.columns if c not in ["unique_id", "ds", "y"]]

X_train_val = None
X_test_future = None
if feature_cols:
    X_train_val = pd.concat([
        train_full[["unique_id", "ds", *feature_cols]],
        val_full[["unique_id", "ds", *feature_cols]]
    ], ignore_index=True).sort_values(["unique_id", "ds"]).reset_index(drop=True).fillna(0)
    X_test_future = test_full[["unique_id", "ds", *feature_cols]].sort_values(["unique_id", "ds"]).reset_index(drop=True).fillna(0)

h_test = get_horizon(futures["test"], splits["test"])
print(f"Train+Val shape: {y_train_val.shape}")
print(f"Test shape: {y_test.shape}")
print(f"Test horizon: {h_test}")
print(f"Feature columns available: {feature_cols}")

Train+Val shape: (19490, 3)
Test shape: (310, 3)
Test horizon: 31
Feature columns available: []


In [23]:
# Train baseline (SeasonalNaive 7-day)
baseline_model = SeasonalNaive(season_length=7)
sf_baseline = StatsForecast(models=[baseline_model], freq=FREQ, n_jobs=-1)
sf_baseline.fit(df=y_train_val)
fcst_baseline = sf_baseline.predict(h=h_test)

baseline_col = [c for c in fcst_baseline.columns if c not in ("unique_id", "ds")][0]
fcst_baseline = fcst_baseline.rename(columns={baseline_col: "baseline_pred"})

# Train best model
best_model_constructor = MODEL_GRID[best_model_name]["constructor"]
best_model = best_model_constructor(**best_params)

if best_model_name == "RandomForestRegressor":
    if X_train_val is None or X_test_future is None:
        raise RuntimeError("RandomForestRegressor selected but no feature columns are available.")
    train_df = y_train_val.merge(X_train_val, on=["unique_id", "ds"], how="left").fillna(0)
    test_df = y_test.merge(X_test_future, on=["unique_id", "ds"], how="left").fillna(0)
    feature_cols_rf = [c for c in train_df.columns if c not in ["unique_id", "ds", "y"]]
    best_model.fit(train_df[feature_cols_rf], train_df["y"])
    test_preds = best_model.predict(test_df[feature_cols_rf])
    fcst_best = test_df[["unique_id", "ds"]].copy()
    fcst_best["best_pred"] = test_preds
else:
    sf_best = StatsForecast(models=[best_model], freq=FREQ, n_jobs=-1)
    fitted_best = sf_best.fit(df=y_train_val)
    fcst_best = fitted_best.predict(h=h_test)
    best_col = [c for c in fcst_best.columns if c not in ("unique_id", "ds")][0]
    fcst_best = fcst_best.rename(columns={best_col: "best_pred"})

print("✓ Baseline and best model trained")

✓ Baseline and best model trained


In [24]:
# Merge predictions with actuals
results_test = y_test.merge(fcst_baseline, on=["unique_id", "ds"], how="left")
results_test = results_test.merge(fcst_best, on=["unique_id", "ds"], how="left")

# Calculate metrics
mae_baseline = mean_absolute_error(results_test["y"], results_test["baseline_pred"])
rmse_baseline = root_mean_squared_error(results_test["y"], results_test["baseline_pred"])
smape_baseline = np.mean(200 * np.abs(results_test["baseline_pred"] - results_test["y"]) / (np.abs(results_test["y"]) + np.abs(results_test["baseline_pred"]) + 1e-8))
r2_baseline = r2_score(results_test["y"], results_test["baseline_pred"])

mae_best = mean_absolute_error(results_test["y"], results_test["best_pred"])
rmse_best = root_mean_squared_error(results_test["y"], results_test["best_pred"])
smape_best = np.mean(200 * np.abs(results_test["best_pred"] - results_test["y"]) / (np.abs(results_test["y"]) + np.abs(results_test["best_pred"]) + 1e-8))
r2_best = r2_score(results_test["y"], results_test["best_pred"])

metrics = pd.DataFrame({
    "model": ["Baseline (SeasonalNaive-7)", f"Best ({best_model_name})"],
    "feature_set": ["none", best_feature],
    "mae": [mae_baseline, mae_best],
    "rmse": [rmse_baseline, rmse_best],
    "smape": [smape_baseline, smape_best],
    "r2": [r2_baseline, r2_best],
})

display(metrics)
metrics

,model,feature_set,mae,rmse,smape,r2
0,Baseline (SeasonalNaive-7),none,100.566129,164.751355,24.294597,0.976751
1,Best (AutoETS),none,86.852207,153.463049,17.512230,0.979828


,model,feature_set,mae,rmse,smape,r2
0,Baseline (SeasonalNaive-7),none,100.566129,164.751355,24.294597,0.976751
1,Best (AutoETS),none,86.852207,153.463049,17.512230,0.979828


In [25]:
# Save predictions and metrics
PRED_DIR = Path("..") / "data" / "predictions"
PRED_DIR.mkdir(parents=True, exist_ok=True)

# Save predictions
results_test_pl = pl.from_pandas(results_test)
pred_path = PRED_DIR / "test_predictions.parquet"
results_test_pl.write_parquet(pred_path)
print(f"Saved predictions to {pred_path}")

# Save metrics
metrics_pl = pl.from_pandas(metrics)
metrics_path = PRED_DIR / "test_metrics.parquet"
metrics_pl.write_parquet(metrics_path)
print(f"Saved metrics to {metrics_path}")

# Save best model configuration
best_config = pd.DataFrame([{
    "model": best_model_name,
    "feature_set": best_feature,
    "params": str(best_params),
    "used_exog": used_exog,
    "val_mae": best_combo["mae"],
    "val_rmse": best_combo["rmse"],
    "val_smape": best_combo["smape"],
    "val_r2": best_combo.get("r2", np.nan),
    "test_mae": mae_best,
    "test_rmse": rmse_best,
    "test_smape": smape_best,
    "test_r2": r2_best,
}])

config_pl = pl.from_pandas(best_config)
config_path = PRED_DIR / "best_model_config.parquet"
config_pl.write_parquet(config_path)
print(f"Saved best model config to {config_path}")

Saved predictions to ..\data\predictions\test_predictions.parquet
Saved metrics to ..\data\predictions\test_metrics.parquet
Saved best model config to ..\data\predictions\best_model_config.parquet


In [31]:
example_1 = pd.read_parquet("../data/predictions/best_model_config.parquet")
example_2 = pd.read_parquet("../data/predictions/test_metrics.parquet")
example_3 = pd.read_parquet("../data/predictions/test_predictions.parquet")
example_4 = pd.read_parquet("../data/predictions/test_metrics_long.parquet")
example_5 = pd.read_parquet("../data/predictions/test_predictions_long.parquet")
example_4.head(), example_5.head()

(                   model_name metrics  preditions
 0  Baseline (SeasonalNaive-7)     mae  100.566129
 1              Best (AutoETS)     mae   86.852207
 2  Baseline (SeasonalNaive-7)    rmse  164.751355
 3              Best (AutoETS)    rmse  153.463049
 4  Baseline (SeasonalNaive-7)   smape   24.294597,
                    model_name     unique_id         ds       y  preditions
 0  Baseline (SeasonalNaive-7)  "Bubble tea" 2025-11-01  1148.0      1382.0
 1  Baseline (SeasonalNaive-7)  "Bubble tea" 2025-11-02  1095.0      1316.0
 2  Baseline (SeasonalNaive-7)  "Bubble tea" 2025-11-03  1097.0      1229.0
 3  Baseline (SeasonalNaive-7)  "Bubble tea" 2025-11-04  1089.0      1429.0
 4  Baseline (SeasonalNaive-7)  "Bubble tea" 2025-11-05  1218.0      1112.0)

## Long-format outputs
Create long-format metrics and predictions and save to data/predictions.

In [30]:
# Create long-format metrics (only numeric metrics)
metric_cols = ["mae", "rmse", "smape", "r2"]
metrics_long = (
    metrics[["model", *metric_cols]]
    .melt(id_vars=["model"], var_name="metrics", value_name="preditions")
    .rename(columns={"model": "model_name"})
)

# Create long-format predictions
baseline_long = results_test[["unique_id", "ds", "y", "baseline_pred"]].rename(columns={"baseline_pred": "preditions"})
baseline_long["model_name"] = "Baseline (SeasonalNaive-7)"
best_long = results_test[["unique_id", "ds", "y", "best_pred"]].rename(columns={"best_pred": "preditions"})
best_long["model_name"] = f"Best ({best_model_name})"
predictions_long = pd.concat([baseline_long, best_long], ignore_index=True)[["model_name", "unique_id", "ds", "y", "preditions"]]

# Save long-format files
metrics_long_pl = pl.from_pandas(metrics_long)
metrics_long_path = PRED_DIR / "test_metrics_long.parquet"
metrics_long_pl.write_parquet(metrics_long_path)
print(f"Saved long metrics to {metrics_long_path}")

predictions_long_pl = pl.from_pandas(predictions_long)
predictions_long_path = PRED_DIR / "test_predictions_long.parquet"
predictions_long_pl.write_parquet(predictions_long_path)
print(f"Saved long predictions to {predictions_long_path}")

Saved long metrics to ..\data\predictions\test_metrics_long.parquet
Saved long predictions to ..\data\predictions\test_predictions_long.parquet
